In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::alabaster==0.7.12=py37_0
  - conda-forge/win-64::altair==3.0.1=py37_0
  - defaults/win-64::anaconda==2018.12=py37_0
  - defaults/win-64::anaconda-client==1.7.2=py37_0
  - defaults/win-64::anaconda-navigator==1.9.6=py37_0
  - defaults/win-64::anaconda-project==0.8.2=py37_0
  - defaults/win-64::asn1crypto==0.24.0=py37_0
  - defaults/win-64::astroid==2.1.0=py37_0
  - defaults/win-64::astropy==3.1=py37he774522_0
  - defaults/win-64::atomicwrites==1.2.1=py37_0
  - defaults/win-64::attrs==18.2.0=py37h28b3542_0
  - defaults/win-64::babel==2.6.0=py37_0
  - defaults/win-64::backcall==0.1.0=py37_0
  - defaults/win-64::backports==1.0=py37_1
  - defaults/win-64::backports.os==0.1.1=py37_0
  - defaults/win-64::backports.shutil_get_terminal_size==1.0.0=py37_2
  - defaults/win-64::bitarray==0.8.3=py37hfa6e2cd_0
  - def


RemoveError: 'setuptools' is a dependency of conda and cannot be removed from
conda's operating environment.



Libraries imported.


In [3]:
rome_hoods=pd.read_csv('rome_hoods.csv')

In [23]:
rome_hoods=rome_hoods[0:22]

In [24]:
rome_hoods=rome_hood.rename(index=str, columns={"Unnamed: 1": "latitude", "Unnamed: 2": "longitude"})

In [27]:
#alredy saved, to not run again

for i in range(0, 20):
    address =  rome_hoods.iloc[i]['Neighbourhood']+', '+ 'Rome'+', '+'Italy'
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
 
    latitude = location.latitude
    longitude = location.longitude

    rome_hoods.iloc[i,1]=latitude
    rome_hoods.iloc[i,2]=longitude

rome_hoods.head() 

,Neighbourhood,latitude,longitude
0,MONTI,41.895813,12.493587
1,TREVI,41.900978,12.483285
2,COLONNA,41.833718,12.753184
3,CAMPO MARZIO,41.904647,12.477055
4,PIGNA,41.897116,12.479196


In [28]:
rome_hoods.iloc[20,1]=41.878352
rome_hoods.iloc[20,2]=12.485295


rome_hoods.iloc[21,1]=41.908078
rome_hoods.iloc[21,2]=12.465706

In [29]:
rome_hoods

,Neighbourhood,latitude,longitude
0,MONTI,41.895813,12.493587
1,TREVI,41.900978,12.483285
2,COLONNA,41.833718,12.753184
3,CAMPO MARZIO,41.904647,12.477055
4,PIGNA,41.897116,12.479196
5,PONTE,42.040346,12.853573
6,PARIONE,41.897358,12.471103
7,REGOLA,41.894375,12.471030
8,S. EUSTACHIO,41.898437,12.475792
9,CAMPITELLI,41.890085,12.487416


In [37]:
latitude=41.895813
longitude=12.493587
map_rome = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(rome_hoods['latitude'], rome_hoods['longitude'], rome_hoods['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rome)  
    
map_rome

In [16]:
rome_hoods.loc[0,'latitude']=latitude

In [45]:
CLIENT_ID = 'NYWXO3QCEVN1ACZZXNGAODACDFN0FU20DD4CG0UUFDFUMSPH' # your Foursquare ID
CLIENT_SECRET = 'SB3U3BFZHI1TB4BULQAUITZB2W12FZ2011NE1MUG4EM4ZHG5' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT=100


Your credentails:
CLIENT_ID: NYWXO3QCEVN1ACZZXNGAODACDFN0FU20DD4CG0UUFDFUMSPH
CLIENT_SECRET:SB3U3BFZHI1TB4BULQAUITZB2W12FZ2011NE1MUG4EM4ZHG5


In [237]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
rome_venues = getNearbyVenues(names=rome_hoods['Neighbourhood'],
                                   latitudes=rome_hoods['latitude'],
                                   longitudes=rome_hoods['longitude']
                                  )


MONTI
TREVI
COLONNA
CAMPO MARZIO
PIGNA
PONTE
PARIONE
REGOLA
S. EUSTACHIO
CAMPITELLI
S. ANGELO
R. XII – RIPA
TRASTEVERE
BORGO
CELIO
ESQUILINO
LUDOVISI
SALLUSTIANO
CASTRO PRETORIO
TESTACCIO
S. SABA
PRATI


In [57]:
rome_venues 

,Neighbourhood,Neighboyrhood Latitude,Neighboyrhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MONTI,41.895813,12.493587,Fatamorgana,41.895610,12.493304,Ice Cream Shop
1,MONTI,41.895813,12.493587,Trieste,41.896305,12.494132,Pizza Place
2,MONTI,41.895813,12.493587,Grezzo,41.896681,12.494535,Pastry Shop
3,MONTI,41.895813,12.493587,Black Market,41.897096,12.494645,Cocktail Bar
4,MONTI,41.895813,12.493587,Analemma Cafe,41.894763,12.491536,Bar
5,MONTI,41.895813,12.493587,Montipalace Hotel,41.895384,12.493839,Hotel
6,MONTI,41.895813,12.493587,Libreria Caffè Bohemien,41.895444,12.492863,Cocktail Bar
7,MONTI,41.895813,12.493587,Aromaticus,41.896840,12.494611,Salad Place
8,MONTI,41.895813,12.493587,Relais Monti,41.896606,12.494637,Bed & Breakfast
9,MONTI,41.895813,12.493587,Gelateria dell'Angeletto,41.894815,12.491431,Ice Cream Shop


In [58]:
# one hot encoding
rome_onehot = pd.get_dummies(rome_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rome_onehot['Neighbourhood'] = rome_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [rome_onehot.columns[-1]] + list(rome_onehot.columns[:-1])
rome_onehot = rome_onehot[fixed_columns]

rome_onehot.head()

,Neighbourhood,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Café,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Embassy / Consulate,Falafel Restaurant,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Monument / Landmark,Multiplex,Music Venue,Paper / Office Supplies Store,Pastry Shop,Peruvian Restaurant,Piano Bar,Pizza Place,Plaza,Pub,Resort,Restaurant,Road,Roman Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Souvenir Shop,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Temple,Theater,Toy / Game Store,Trattoria/Osteria,Wine Bar,Wine Shop,Women's Store
0,MONTI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,MONTI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,MONTI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,MONTI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,MONTI,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
rome_grouped = rome_onehot.groupby('Neighbourhood').mean().reset_index()
rome_grouped

,Neighbourhood,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Café,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Embassy / Consulate,Falafel Restaurant,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Monument / Landmark,Multiplex,Music Venue,Paper / Office Supplies Store,Pastry Shop,Peruvian Restaurant,Piano Bar,Pizza Place,Plaza,Pub,Resort,Restaurant,Road,Roman Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Souvenir Shop,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Temple,Theater,Toy / Game Store,Trattoria/Osteria,Wine Bar,Wine Shop,Women's Store
0,BORGO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CAMPITELLI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.529412,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.235294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,CAMPO MARZIO,0.000000,0.029412,0.000000,0.000000,0.000000,0.029412,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.029412,0.000000,0.029412,0.000000,0.000000,0.147059,0.029412,0.029412,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.029412,0.000000,0.000000,0.

lets try to put together new york data

In [90]:
manhattan_hoods=pd.read_csv('manhattan_hoods.csv')

In [91]:
manhattan_hoods.head()

,Neighbourhood,latitude,longitude
0,Marble Hill,40.876551,-73.910660
1,Chinatown,40.715618,-73.994279
2,Washington Heights,40.851903,-73.936900
3,Inwood,40.867684,-73.921210
4,Hamilton Heights,40.823604,-73.949688


In [98]:
latitude=40.7623
longitude=-73.9795
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(manhattan_hoods['latitude'], manhattan_hoods['longitude'], manhattan_hoods['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [99]:
manhattan_venues = getNearbyVenues(names=manhattan_hoods['Neighbourhood'],
                                   latitudes=manhattan_hoods['latitude'],
                                   longitudes=manhattan_hoods['longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town


create a unique dataframe

In [101]:
manhattan_venues.insert(0, 'City', 'New York')
rome_venues.insert(0, 'City', 'Rome')

In [103]:
manhattan_venues.head()

,City,Neighbourhood,Neighboyrhood Latitude,Neighboyrhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,New York,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
2,New York,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
3,New York,Marble Hill,40.876551,-73.91066,CTown Supermarkets,40.876218,-73.908541,Supermarket
4,New York,Marble Hill,40.876551,-73.91066,Terrace View Delicatessen,40.876476,-73.912746,Deli / Bodega


In [104]:
rome_venues.head()

,City,Neighbourhood,Neighboyrhood Latitude,Neighboyrhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rome,MONTI,41.895813,12.493587,Fatamorgana,41.895610,12.493304,Ice Cream Shop
1,Rome,MONTI,41.895813,12.493587,Trieste,41.896305,12.494132,Pizza Place
2,Rome,MONTI,41.895813,12.493587,Grezzo,41.896681,12.494535,Pastry Shop
3,Rome,MONTI,41.895813,12.493587,Black Market,41.897096,12.494645,Cocktail Bar
4,Rome,MONTI,41.895813,12.493587,Analemma Cafe,41.894763,12.491536,Bar


In [105]:
df_2Cities=manhattan_venues.append(rome_venues, ignore_index=True)

In [107]:
df_2Cities.to_csv('NewYork_Rome_venues')

In [112]:
# one hot encoding
onehot = pd.get_dummies(df_2Cities[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#onehot['Neighbourhood'] = df_2Cities['Neighbourhood'] 
#onehot['City'] = df_2Cities['City'] 
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
onehot.insert(0, 'City', df_2Cities['City'])
onehot.insert(1, 'Neighbourhood', df_2Cities['Neighbourhood'])
onehot.head()

,City,Neighbourhood,Yoga Studio,Abruzzo Restaurant,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roman Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy

In [117]:
df_2cities_grouped = onehot.groupby(['City','Neighbourhood']).mean().reset_index()
df_2cities_grouped

,City,Neighbourhood,Yoga Studio,Abruzzo Restaurant,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roman Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy

In [174]:
num_top_venues = 5

for hood in df_2cities_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = df_2cities_grouped[df_2cities_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp=temp.drop([0,1]).reset_index()
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
   index             venue  freq
0    200              Park  0.13
1     76      Cupcake Shop  0.06
2    234    Sandwich Place  0.06
3    108        Food Court  0.06
4     81  Department Store  0.06


----Carnegie Hill----
   index                 venue  freq
0    131  Gym / Fitness Center  0.08
1     51                  Café  0.08
2    249                   Spa  0.06
3    154    Italian Restaurant  0.06
4    130                   Gym  0.06


----Central Harlem----
   index                            venue  freq
0     73                   Cosmetics Shop  0.10
1    189                      Music Venue  0.05
2     51                             Café  0.05
3    112                French Restaurant  0.05
4    247  Southern / Soul Food Restaurant  0.05


----Chelsea----
   index               venue  freq
0     51                Café  0.06
1    193           Nightclub  0.06
2    251           Speakeasy  0.06
3    178  Mexican Restaurant  0.06
4    146               H

   index                venue  freq
0     73       Cosmetics Shop  0.07
1    154   Italian Restaurant  0.06
2     65          Coffee Shop  0.05
3      8  American Restaurant  0.05
4    119            Gastropub  0.04


----Yorkville----
   index             venue  freq
0    288         Wine Shop  0.09
1    260  Sushi Restaurant  0.09
2     85             Diner  0.05
3     73    Cosmetics Shop  0.05
4     22              Bank  0.05


----BORGO----
   index                          venue  freq
0    154             Italian Restaurant  0.75
1    189                    Music Venue  0.25
2      3                    Yoga Studio  0.00
3    193                      Nightclub  0.00
4    199  Paper / Office Supplies Store  0.00


----CAMPITELLI----
   index                venue  freq
0    140        Historic Site  0.53
1    268               Temple  0.24
2    183  Monument / Landmark  0.06
3    116               Garden  0.06
4    141       History Museum  0.06


----CAMPO MARZIO----
   index      

In [178]:
def return_most_common_venues(row, num_top_venues):
   # temp=temp.drop([0,1]).reset_index()
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [188]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = df_2cities_grouped['Neighbourhood']
for ind in np.arange(df_2cities_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_2cities_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Cupcake Shop,Department Store,Food Court,Sandwich Place,Performing Arts Venue,Plaza,Coffee Shop,Salad Place,Bakery
1,Carnegie Hill,Café,Gym / Fitness Center,Spa,Italian Restaurant,Gym,Shoe Store,Korean Restaurant,Sports Bar,Bookstore,Pharmacy
2,Central Harlem,Cosmetics Shop,Juice Bar,Deli / Bodega,Dessert Shop,Ethiopian Restaurant,Caribbean Restaurant,Music Venue,Café,Southern / Soul Food Restaurant,Breakfast Spot
3,Chelsea,Hotel,Café,Speakeasy,Nightclub,Mexican Restaurant,Beer Bar,Liquor Store,French Restaurant,Mediterranean Restaurant,Men's Store
4,Chinatown,Bubble Tea Shop,Chinese Restaurant,Spa,Korean Restaurant,Hotpot Restaurant,Noodle House,Japanese Restaurant,Sandwich Place,Vietnamese Restaurant,Cantonese Restaurant


CLUSTERING

In [183]:
kclusters = 5

grouped_clustering = df_2cities_grouped.drop('Neighbourhood', 1)
grouped_clustering = grouped_clustering.drop('City', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4,
       3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 1, 3])

In [347]:
#manhattan_hoods.insert(0, 'city', 'New York')
#rome_hoods.insert(0, 'city', 'Rome')
data_tot = data_tot.iloc[0:0]
data_tot=manhattan_hoods.append(rome_hoods,ignore_index=True)
data_tot.head()

,city,Neighbourhood,latitude,longitude
0,New York,Marble Hill,40.876551,-73.910660
1,New York,Chinatown,40.715618,-73.994279
2,New York,Washington Heights,40.851903,-73.936900
3,New York,Inwood,40.867684,-73.921210
4,New York,Hamilton Heights,40.823604,-73.949688


In [211]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#merged = data_tot

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#merged = merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

merged=merged.drop([43,48])
merged
# check the last columns!

,city,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Marble Hill,40.876551,-73.910660,0.0,Sandwich Place,Pizza Place,Deli / Bodega,Pharmacy,Supermarket,Train Station,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Embassy / Consulate
1,New York,Chinatown,40.715618,-73.994279,1.0,Bubble Tea Shop,Chinese Restaurant,Spa,Korean Restaurant,Hotpot Restaurant,Noodle House,Japanese Restaurant,Sandwich Place,Vietnamese Restaurant,Cantonese Restaurant
2,New York,Washington Heights,40.851903,-73.936900,1.0,Café,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Park,Pizza Place,Grocery Store,Bakery,Fast Food Restaurant,Mexican Restaurant
3,New York,Inwood,40.867684,-73.921210,1.0,Café,Pizza Place,Restaurant,Frozen Yogurt Shop,Bakery,American Restaurant,Bank,Diner,Wine Bar,Deli / Bodega
4,New York,Hamilton Heights,40.823604,-73.949688,1.0,Mexican Restaurant,Yoga Studio,Cocktail Bar,Deli / Bodega,Coffee Shop,Pub,Caribbean Restaurant,Café,Bar,Cosmetics Shop
5,New York,Manhattanville,40.816934,-73.957385,1.0,Coffee Shop,Bank,Bus Station,Climbing Gym,Gas Station,Juice Bar,Japanese Curry Restaurant,Sandwich Place,Café,Music School
6,New York,Central Harlem,40.815976,-73.943211,1.0,Cosmetics Shop,Juice Bar,Deli / Bodega,Dessert Shop,Ethiopian Restaurant,Caribbean Restaurant,Music Venue,Café,Southern / Soul Food Restaurant,Breakfast Spot
7,New York,East Harlem,40.792249,-73.944182,1.0,Pizza Place,Latin American Restaurant,Fried Chicken Joint,Bank,Clothing Store,Donut Shop,Diner,Shoe Store,Spanish Restaurant,Street Art
8,New York,Upper East Side,40.775639,-73.960508,1.0,Juice Bar,Boutique,Coffee Shop,Italian Restaurant,Miscellaneous Shop,Burrito Place,Snack Place,Bridal Shop,Pet Store,Café
9,New York,Yorkville,40.775930,-73.947118,1.0,Wine Shop,Sushi Restaurant,Cosmetics Shop,Gourmet Shop,Deli / Bodega,Hobby Shop,Pharmacy,Diner,Coffee Shop,Café


In [212]:
latitude=40.7623
longitude=-73.9795
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'],merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [213]:
latitude=41.895813
longitude=12.493587
map_clusters1 = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'],merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

To make things more interensting,m let's add Toronto, and see if it is more similar to New york or rome 

In [214]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [215]:
df= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [217]:
df1=df[df.Borough != 'Not assigned']
df3=df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
pd.DataFrame(df3)
df3.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [219]:
indexes=np.where(df3["Neighbourhood"] == 'Not assigned')
for i in indexes:
    df3.iloc[i,2]=df3.iloc[i,1]
geo_locations_df=pd.read_csv('http://cocl.us/Geospatial_data')
df4=df3.join(geo_locations_df[['Latitude','Longitude']])
df4.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


As we did for New York and Roe, let's consider only the city center, that should be Toronto Downtown

In [249]:
def getNearbyVenues(borough, names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for borough,name, lat, lng in zip(borough,names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print("RESULTS:", results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [250]:

toronto_data=df4
borough =toronto_data['Borough']
names =toronto_data['Neighbourhood']
latitudes=toronto_data['Latitude']
longitudes=toronto_data['Longitude']

toronto_venues = getNearbyVenues(borough,names,latitudes,longitudes)

In [262]:
#toronto_venues.insert(0, 'City', 'Toronto')

toronto_venues = toronto_venues[toronto_venues['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_venues=toronto_venues.drop(columns=['Borough'])
toronto_venues.insert(0, 'City', 'Toronto')
toronto_venues.head()

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Toronto,Rosedale,43.679563,-79.377529,Park Drive Reservation Lands,43.679822,-79.377787,Park
1,Toronto,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
2,Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
3,Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
4,Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant


In [278]:
df_2Cities=df_2Cities.rename(index=str, columns={"Neighbourhood": "Neighborhood", "Neighboyrhood Latitude": "Neighborhood Latitude","Neighboyrhood Longitude": "Neighborhood Longitude"})
df_3Cities=df_2Cities.append(toronto_venues, ignore_index=True)

In [282]:
df_3Cities.head()

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,New York,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
2,New York,Marble Hill,40.876551,-73.91066,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
3,New York,Marble Hill,40.876551,-73.91066,CTown Supermarkets,40.876218,-73.908541,Supermarket
4,New York,Marble Hill,40.876551,-73.91066,Terrace View Delicatessen,40.876476,-73.912746,Deli / Bodega


In [306]:
onehotN = pd.get_dummies(df_3Cities[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#onehot['Neighbourhood'] = df_2Cities['Neighbourhood'] 
#onehot['City'] = df_2Cities['City'] 
# move neighborhood column to the first column
#fixed_columns = [onehotnew.columns[-1]] + list(onehotnew.columns[:-1])
#onehotnew = onehotnew[fixed_columns]
#onehotnew.insert(0, 'City', df_3Cities['City'])
#onehotnew.insert(1, 'Neighborhood', df_3Cities['Neighborhood'])
#onehotnew.head()

In [320]:
#onehotN.insert(0, 'City', df_3Cities['City'])
onehotN=onehotN.drop(columns= 'Neighborhood')

In [321]:
onehotN.insert(1, 'Neighborhood', df_3Cities['Neighborhood'])
onehotN.head()

,City,Neighborhood,Abruzzo Restaurant,Accessories Store,Adult Boutique,Afghan Restaurant,Airport Lounge,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roman Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Sw

In [322]:
df_3cities_grouped = onehotN.groupby(['City','Neighborhood']).mean().reset_index()
df_3cities_grouped

,City,Neighborhood,Abruzzo Restaurant,Accessories Store,Adult Boutique,Afghan Restaurant,Airport Lounge,Airport Service,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roman Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Sw

In [323]:
def return_most_common_venues(row, num_top_venues):
   # temp=temp.drop([0,1]).reset_index()
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [327]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = df_3cities_grouped['Neighborhood']
for ind in np.arange(df_3cities_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(df_3cities_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Department Store,Cupcake Shop,Food Court,Sandwich Place,Pharmacy,Coffee Shop,Smoke Shop,Gastropub,Salad Place
1,Carnegie Hill,Café,Gym / Fitness Center,Italian Restaurant,Gym,Spa,Dance Studio,Deli / Bodega,Korean Restaurant,Sports Bar,Pizza Place
2,Central Harlem,Cosmetics Shop,Breakfast Spot,Bagel Shop,Cycle Studio,Café,French Restaurant,Fried Chicken Joint,Music Venue,Lounge,Caribbean Restaurant
3,Chelsea,Mexican Restaurant,Nightclub,Café,Hotel,Speakeasy,Cupcake Shop,Asian Restaurant,Liquor Store,Coffee Shop,Event Space
4,Chinatown,Bubble Tea Shop,Chinese Restaurant,Japanese Restaurant,Hotpot Restaurant,Sandwich Place,Vietnamese Restaurant,Korean Restaurant,Spa,Noodle House,Record Shop


In [372]:
kclusters = 5

grouped_clustering1 = df_3cities_grouped.drop('Neighborhood', 1)
grouped_clustering1 = grouped_clustering1.drop('City', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 0, 0, 0, 1,
       0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1])

In [344]:
Downtown_data = df4[df4['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_data=Downtown_data.drop(columns='Postcode')
Downtown_data=Downtown_data.drop(columns='Borough')
Downtown_data.insert(0, 'city', 'Toronto')
Downtown_data=Downtown_data.rename(index=str, columns={"City": "city", "Latitude": "latitude","Longitude": "longitude"})
Downtown_data.head()

,city,Neighbourhood,latitude,longitude
0,Toronto,Rosedale,43.679563,-79.377529
1,Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,Toronto,Church and Wellesley,43.665860,-79.383160
3,Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [375]:
labels= kmeans.labels_
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', labels)

#np.delete(kmeans.labels_,[43,48])


In [376]:
# add clustering labels
#neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans.labels_)

merged1 = data_tot.drop([43,48])

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged1 = merged1.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighbourhood')

#merged=merged.drop([43,48])
merged1

,city,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Pizza Place,Supermarket,Pharmacy,Deli / Bodega,Train Station,Event Space,Duty-free Shop,Eastern European Restaurant,Electronics Store
1,New York,Chinatown,40.715618,-73.994279,1,Bubble Tea Shop,Chinese Restaurant,Japanese Restaurant,Hotpot Restaurant,Sandwich Place,Vietnamese Restaurant,Korean Restaurant,Spa,Noodle House,Record Shop
2,New York,Washington Heights,40.851903,-73.936900,1,Café,Chinese Restaurant,Pizza Place,Grocery Store,Bakery,Mobile Phone Shop,Deli / Bodega,Park,Accessories Store,Breakfast Spot
3,New York,Inwood,40.867684,-73.921210,1,Café,Diner,Pizza Place,Bakery,Wine Bar,Bank,American Restaurant,Restaurant,Frozen Yogurt Shop,Chinese Restaurant
4,New York,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Yoga Studio,Bar,Deli / Bodega,Coffee Shop,Cocktail Bar,Caribbean Restaurant,Café,Pub,Wine Bar
5,New York,Manhattanville,40.816934,-73.957385,1,Coffee Shop,Sandwich Place,Climbing Gym,Gas Station,Deli / Bodega,Ramen Restaurant,Pharmacy,Bus Station,Music School,Café
6,New York,Central Harlem,40.815976,-73.943211,1,Cosmetics Shop,Breakfast Spot,Bagel Shop,Cycle Studio,Café,French Restaurant,Fried Chicken Joint,Music Venue,Lounge,Caribbean Restaurant
7,New York,East Harlem,40.792249,-73.944182,1,Pizza Place,Latin American Restaurant,Fried Chicken Joint,Clothing Store,Bank,Mobile Phone Shop,Park,Beer Bar,Donut Shop,Bakery
8,New York,Upper East Side,40.775639,-73.960508,1,Juice Bar,Boutique,Coffee Shop,Bridal Shop,Café,Snack Place,Burger Joint,Burrito Place,Miscellaneous Shop,Shoe Store
9,New York,Yorkville,40.775930,-73.947118,1,Sushi Restaurant,Wine Shop,Hobby Shop,Gym,Liquor Store,Coffee Shop,Bar,Bank,Bagel Shop,Gourmet Shop


In [377]:
#new york map
latitude=40.7623
longitude=-73.9795
map_clusters_NewYork = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged1['latitude'], merged1['longitude'],merged1['Neighbourhood'], merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters_NewYork)
       
map_clusters_NewYork

In [380]:
latitude=43.665860
longitude=-79.383160
map_clusters_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged1['latitude'], merged1['longitude'],merged1['Neighbourhood'], merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters_Toronto)
       
map_clusters_Toronto

In [381]:
latitude=41.895813
longitude=12.493587
map_clusters_Rome = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged1['latitude'], merged1['longitude'],merged1['Neighbourhood'], merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters_Rome)
       
map_clusters_Rome